# Important notes:
    if you wanna run this code make sure that minNeighbors and scaleFactor fit your conditions (light, brightness, camsize etc)
    for example                                 minNeighbors, scaleFactor
    masks = mask_cascade.detectMultiScale(gray,     1.34    ,     18     ) 
    if you wanna see an example video of runs (model 4-6) after i optimized those weights you can go to dir ./openMe/...
    also added vid that shows the features chosen for model 4

    need to change with these since too low vals gives you alot of false Positives
    and too high give false Negatives

    the color of your mask is not the same color as the background (black mask on dark background doesnt give bad results and not included in my test). 
    my test is 200 photos (100 pos, 100 neg) i took by myself using my webcan wearing blue standat mask at night (room's light).

    for the 4th, 5th and 6th models there are videos that demonstrate real-time use at the folder OPEN_ME in this task folder.
    its better then if you try to use urself since youll have to adjust  minNeighbors, scaleFactor to make model work well.

    I ran this project on my Ubunto VM. therefor not every print is here, but i mentioned the results.
    
    There are no grapths of the learning curves since plt.plot_model function support only objects that support .fit command, so if we wanna draw we have to create the model of 1 stages from ubunto (~15 min) + create the model of 2 stages from ubunto (~35 min) + .... + create the model of 25 stages from ubunto (~240 min) this work is hard and consumes too much time, and without knowlege in scripts we have to type commands menually each time (traincastade -stages=1, traincastade -stages=2, ..., traincastade -stages=25)

handeling the data
--------------------------

    copy the basic imports from CNN solution to handle the data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os
import zipfile
import random
import shutil
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg

In [ ]:
anno_dir ='F:\\MLBig\\Creating_VJ_Cascade\\Medical Mask\\annotations'
images_dir = 'F:\\MLBig\\Creating_VJ_Cascade\\Medical Mask\\images'

    code added to create info.dat / bg.txt 

In [ ]:
images=[]
labels=[]

f=open('info.dat', 'w')
fb = open('bg.txt', 'w')
masked_img  = False
for filename in os.listdir(images_dir):
    num = filename.split('.')[ 0 ]
    print("loading image: {}".format(filename))
    if int(num) > 1800:
        class_name = None
        anno = filename + ".json"
        with open(os.path.join(anno_dir, anno)) as json_file:
            json_data = json.load(json_file)
            no_anno = json_data["NumOfAnno"]
            k = 0
            for i in range(0, no_anno):
                class_nam = json_data['Annotations'][i]['classname']
                if class_nam in ['face_with_mask',"gas_mask", "face_shield", "mask_surgical", "mask_colorful"]:
                    class_name = 'face_with_mask'
                    k = i
                    masked_img = True
                    break
                elif class_nam in ['face_no_mask,"hijab_niqab', 'face_other_covering', "face_with_mask_incorrect", "scarf_bandana", "balaclava_ski_mask", "other" ]:
                    class_name = 'face_no_mask'
                    k = i
                    masked_img = False
                    break
                else:
                    continue
                    
            box = json_data[ 'Annotations' ][k][ 'BoundingBox' ]
            (x1, x2, y1, y2) = box
            print('point:'+str(x1)+','+str(x2)+','+str(y1)+','+str(y2))
            if masked_img:
                f.write('Medical Mask\\images\\' + filename + ' 1 ' + str(x1) + ' ' + str(x2) + ' ' +  str(y1) + ' ' + str(y2) + ' \n')
            else:
                fb.write('Medical Mask\\images\\' + filename + ' \n')
        if class_name is not None: # we ignore some faces here
            image = cv2.imread(os.path.join(images_dir, filename))
            img = image[x2:y2, x1:y1]
            img = cv2.resize(img, (224, 224))
            img = img[...,::-1].astype(np.float32)
            img = preprocess_input(img)
            images.append(img)
            labels.append(class_name)  
   
f.close()
fb.close()
images = np.array(images, dtype="float32")
labels = np.array(labels)
print(len(images))
print(len(labels))

FIRST ATTEMT TO CREATE haar cascade classifier for faceMask detection
----------------------------------------------------------------------------------------------------------

    we create pos and neg dirs for the data convert it to gray and cut the faces with our labels (only for pos since anything is really negative)

In [ ]:
import cv2
import numpy as np
import os
from csv import reader




def store_raw_images():
    pic_num = 1
    name = ""    
    if not os.path.exists('pos'):
        os.makedirs('pos')
        
    with open("info.dat",'r') as read_obj:
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # Iterate over each row in the csv using reader object
        for i in csv_reader:
            splitted = i[0].split(" ")
            srcPathName = splitted[0] + " " + splitted[1]
            num_of_obj = splitted[2]
            x1 = splitted[3]
            x2 = splitted[4]  
            y1 = splitted[5]
            y2 = splitted[6]
            for j in i[0].split("."):
                i[0] = i[0].strip()
                j=j[0:3] # remove space
                #.xxx extensions
                if pic_num == 1: # cus i testet first
                    pic_num += 1
                    continue
                if j not in ['jpg', 'png']:
                    continue
                try:
                    print(i[0])
                    if j == 'jpg':
                        name = "pos/"+str(pic_num)+".jpg"
                        os.rename(srcPathName, name)
                    else:
                        name = "pos/"+str(pic_num)+".png"
                        os.rename(srcPathName, name)
                    img = cv2.imread(name,cv2.IMREAD_GRAYSCALE)
                    img = img[int(x2):int(y2), int(x1):int(y1)]
                    # should be larger than samples / pos pic (so we can place our image on it)
                    resized_image = cv2.resize(img, (50, 50))
                    os.remove(name)
                    cv2.imwrite(name,resized_image)
                    print(pic_num)
                    pic_num += 1
                
                except Exception as e:
                    print(str(e))  



store_raw_images()

In [ ]:
import cv2
import numpy as np
import os
from csv import reader




def store_raw_images():
    neg_image_urls = open("bg.txt",'r')
    pic_num = 1
    name = ""    
    if not os.path.exists('neg'):
        os.makedirs('neg')
        
    with open("bg.txt",'r') as read_obj:
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # Iterate over each row in the csv using reader object
        for i in csv_reader:
            for j in i[0].split("."):
                i[0] = i[0].strip()
                j=j.strip() # remove space
                if pic_num == 1: # cus i testet first
                    pic_num += 1
                    continue
                if len(j)>4:
                    continue
                try:
                    print(i[0])
                    if j == 'jpg':
                        name = "neg/"+str(pic_num)+".jpg"
                        os.rename(i[0], name)
                    else:
                        name = "neg/"+str(pic_num)+".png"
                        os.rename(i[0], name)
                    img = cv2.imread(name,cv2.IMREAD_GRAYSCALE)
                    # should be larger than samples / pos pic (so we can place our image on it)
                    resized_image = cv2.resize(img, (100, 100))
                    os.remove(name)
                    cv2.imwrite(name,resized_image)
                    print(pic_num)
                    pic_num += 1
                
                except Exception as e:
                    print(str(e))  



store_raw_images()

    Then we update bg / info files to create our haar cascade with the new data

In [ ]:
import os

f=open('info.dat', 'w')
def update_info():
    pos_images_dir = 'F:\MLBig\Creating_VJ_Cascade\pos'
    for filename in os.listdir(pos_images_dir):
        num = filename.split('.')[ 0 ]
        # first param how many objects then x,y,x+w,y+h
        #since we cut face its the entire img
        f.write('pos/' + filename + ' 1 0 0 50 50 \n')
        print(num)
    f.close()

update_info()


In [ ]:
import os

f=open('bg.txt', 'w')
def update_bg():
    neg_images_dir = 'F:\\MLBig\\Creating_VJ_Cascade\\neg'
    for filename in os.listdir(neg_images_dir):
        num = filename.split('.')[ 0 ]
        f.write('neg/' + filename + ' \n')
        print(num)
    f.close()

update_bg()

    we train in Linux according to the guide https://pythonprogramming.net/haar-cascade-object-detection-python-opencv-tutorial/
    (opencv_traincascade command in linux)
    the following code opens your camera and draw rect in diffrent colors if ur wearing mask / no mask

In [ ]:
import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#this is the cascade we just made. Call what you want
mask_cascade = cv2.CascadeClassifier('mask_cascade_1.xml')

cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    maskes = mask_cascade.detectMultiScale(gray,1.3,20)# 1.3,20
    
    # add this


    for (x,y,w,h) in faces:
        print("("+str(x)+','+str(y)+'),('+str(x+w)+','+str(y+h)+')')
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in maskes:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)


    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

    I get alot of falses and I dont like this solution.
    Therefore we try again training on 224x224 images.
    increasing the stages from 10 to 15
    I'm not testing this model since its really not accurate.

SECOND ATTEMT TO CREATE haar cascade classifier for faceMask detection
----------------------------------------------------------------------------------------------------------

    we changed the generate data code to fit this specific model here.

In [ ]:
images=[]
labels=[]

f=open('info.dat', 'w')
fb = open('bg.txt', 'w')
poscounter=1
negcounter=1
masked_img  = False
for filename in os.listdir(images_dir):
    num = filename.split('.')[ 0 ]
    print("loading image: {}".format(filename))
    if int(num) > 1800:
        class_name = None
        anno = filename + ".json"
        with open(os.path.join(anno_dir, anno)) as json_file:
            json_data = json.load(json_file)
            no_anno = json_data["NumOfAnno"]
            k = 0
            for i in range(0, no_anno):
                class_nam = json_data['Annotations'][i]['classname']
                if class_nam in ['face_with_mask', "mask_surgical", "mask_colorful"]:
                    class_name = 'face_with_mask'
                    k = i
                    masked_img = True
                    break
                elif class_nam in ['face_no_mask,"hijab_niqab', 'face_other_covering', "face_with_mask_incorrect", "scarf_bandana","balaclava_ski_mask", "other" ]:                   
                    class_name = 'face_no_mask'
                    k = i
                    masked_img = False
                    break
                else:
                    continue
                    
            box = json_data[ 'Annotations' ][k][ 'BoundingBox' ]
            (x1, x2, y1, y2) = box
            print('point:'+str(x1)+','+str(x2)+','+str(y1)+','+str(y2))
            # pathes on linux syntax since train is on linux.
            if masked_img:
                #f.write('pos/' + filename + ' 1 ' + str(x1) + ' ' + str(x2) + ' ' +  str(y1) + ' ' + str(y2) + ' \n')
                f.write('pos/' + str(poscounter) + ' 1 0 0 224 224 \n')
            else:
                fb.write('neg/' + str(negcounter) + ' \n')
        if class_name is not None: # we ignore some faces here
            image = cv2.imread(os.path.join(images_dir, filename),cv2.IMREAD_GRAYSCALE)
            img = image[x2:y2, x1:y1]
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(class_name)  
            if masked_img:
                if filename.split('.')[ 1 ] == 'jpeg':
                    print('pos\\'+str(poscounter)+"."+filename.split('.')[ 1 ])
                    cv2.imwrite('pos\\'+str(poscounter)+"."+'jpg',img)
                    poscounter += 1
                else:
                    print('pos\\'+str(poscounter)+"."+filename.split('.')[ 1 ])
                    cv2.imwrite('pos\\'+str(poscounter)+"."+filename.split('.')[ 1 ],img)
                    poscounter += 1
            else:
                if filename.split('.')[ 1 ] == 'jpeg':
                    print('neg\\'+str(negcounter)+"."+filename.split('.')[ 1 ])
                    cv2.imwrite('neg\\'+str(negcounter)+"."+'jpg',img)
                    negcounter += 1
                else:
                    print('neg\\'+str(negcounter)+"."+filename.split('.')[ 1 ])
                    cv2.imwrite('neg\\'+str(negcounter)+"."+filename.split('.')[ 1 ],img)
                    negcounter += 1

   
f.close()
fb.close()
images = np.array(images, dtype="float32")
labels = np.array(labels)
print(len(images))
print(len(labels))

    we train in Linux according to the guide https://pythonprogramming.net/haar-cascade-object-detection-python-opencv-tutorial/
    the following code opens your camera and draw rect in diffrent colors if ur wearing mask / no mask

In [ ]:
import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#this is the cascade we just made. Call what you want
watch_cascade = cv2.CascadeClassifier('mask_cascade_2.xml')

cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    watches = watch_cascade.detectMultiScale(gray,1.3,20)# 1.3,20
    
    # add this


    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in watches:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)


    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

    better but still not good enough
    model evaluation: 100 pos images + 100 neg images through my webcam at night

In [ ]:
from typing import Counter
import numpy as np
import cv2
import time

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#this is the cascade we just made. Call what you want
mask_cascade = cv2.CascadeClassifier('mask_cascade_2.xml')

cap = cv2.VideoCapture(0)
pos_counter = 0
iter_counter = 0
neg_counter = 0

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    """""
    Try diffrent Params in diff light / brightness / face size (if far or close)
    """
    masks = mask_cascade.detectMultiScale(gray,1.34,18) 
    #FOR RAB:
    #1.3, 15 WORK WELL
    
    #1.3 19 Seems best in tomer's conditions at night.
    #1.3,11 DOESNT WORK FOR UNCOVER NOSE WELL BUT DECENT
    #GAB 1.3 19 Seems decent (PROB NOT RAB)
    
    # GAB
    # 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this
    masked = True

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        masked = False


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        masked = True
        cv2.putText(img,'Mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 0),4)#((x*2+w)/2,(y*2+h)/2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    # if haarcascade_frontalface_default.xml detects face but our didnt we put txt no mask
    if not masked:
        cv2.putText(img,'No mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),4)#((x*2+w)/2,(y*2+h)/2)
    iter_counter += 1 
    if masked:
        pos_counter += 1
    if iter_counter == 100:
        break
    print(str(iter_counter))
    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

time.sleep(10)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    """""
    Try diffrent Params in diff light / brightness / face size (if far or close)
    """
    masks = mask_cascade.detectMultiScale(gray,1.3,19) 
    #FOR RAB:
    #1.3, 15 WORK WELL
    
    #1.3 19 Seems best in tomer's conditions at night.
    #1.3,11 DOESNT WORK FOR UNCOVER NOSE WELL BUT DECENT
    #GAB 1.3 19 Seems decent (PROB NOT RAB)
    
    # GAB
    # 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this
    masked = True

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        masked = False


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        masked = True
        cv2.putText(img,'Mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 0),4)#((x*2+w)/2,(y*2+h)/2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    # if haarcascade_frontalface_default.xml detects face but our didnt we put txt no mask
    if not masked:
        cv2.putText(img,'No mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),4)#((x*2+w)/2,(y*2+h)/2)
    iter_counter += 1 
    if not masked:
        neg_counter += 1
    if iter_counter == 200:
        break
    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    print(str(iter_counter))

score = (neg_counter+pos_counter)/iter_counter
print(score)
cap.release()
cv2.destroyAllWindows()

TEST ACC: 0.88

THIRD ATTEMT TO CREATE haar cascade classifier for faceMask detection
----------------------------------------------------------------------------------------------------------

    here were increasing to 15 stages
    files size increased to 224_224
    the training is on 120 width 100 height to fit my camera
    Ofc we handle the data first I didnt changed the code so i didnt coppied it

    You can skip this note

    (Some Numerical info and why I chose these params:)

"""""
Conclusion:

cascade tarined as HAAR

-v-HAAR classifiers are very accurate but require a lot more time to train
-x-LBP classifiers on the other hand are less accurate but train much quicker and detect almost 3 times faster.

-mode <BASIC (default) | CORE | ALL> : Selects the type of Haar features set used in training. BASIC use only upright features, while ALL uses the full set of upright and 45 degree rotated feature set. See [148] for more details.
---> BASIC done, try ALL

Adaboost PARAMS
---> value + explain:

-acceptanceRatioBreakValue <break_value> : This argument is used to determine how precise your model should keep learning and when to stop. A good guideline is to train not further than 10e-5, to ensure the model does not overtrain on your training data. By default this value is set to -1 to disable this feature.
---> -1, we dont need this since we dont have unlimited time.

-Boost type: GAB (Gentle Adaboost)
for params explain: https://topic.alibabacloud.com/a/comparison-of-several-boost-algorithms-discrete-adaboost-real-adaboost-logitboost-gentle-adaboost-__-machine-learning_8_8_20288852.html
-weightTrimRate <weight_trim_rate> : Specifies whether trimming should be used and its weight. A decent choice is 0.95.
---> 0.95
-maxDepth <max_depth_of_weak_tree> : Maximal depth of a weak tree. A decent choice is 1, that is case of stumps.
---> 1
-maxFalseAlarmRate <max_false_alarm_rate> : Maximal desired false alarm rate for each stage of the classifier. Overall false alarm rate may be estimated as (max_false_alarm_rate ^ number_of_stages)
---> 0.5^15 / 0.5^25 (FA - 0.5, Stages - 15 / 25) small nums decent choise (may wanna try lower if i have more time)
-maxWeakCount <max_weak_tree_count> : Maximal count of weak trees for every cascade stage. The boosted classifier (stage) will have so many weak trees (<=maxWeakCount), as needed to achieve the given -maxFalseAlarmRate.
---> 100 by default choise (may wanna try more options if i have more time)

neg img: 1082
pos img: 69% * 2600

changed: 224x224 train pic the detection is 24x24

"""


In [ ]:
import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#this is the cascade we just made. Call what you want
mask_cascade = cv2.CascadeClassifier('mask_cascade.xml')

cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    masks = mask_cascade.detectMultiScale(gray,1.6,25)# 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this


    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)


    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

    this starts to look decent and we think about where we still wanna improve now
    lets evaluate the model:

In [ ]:
from typing import Counter
import numpy as np
import cv2
import time

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#this is the cascade we just made. Call what you want
mask_cascade = cv2.CascadeClassifier('mask_cascade.xml')

cap = cv2.VideoCapture(0)
pos_counter = 0
iter_counter = 0
neg_counter = 0

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    """""
    Try diffrent Params in diff light / brightness / face size (if far or close)
    """
    masks = mask_cascade.detectMultiScale(gray,1.34,18) 
    #FOR RAB:
    #1.3, 15 WORK WELL
    
    #1.3 19 Seems best in tomer's conditions at night.
    #1.3,11 DOESNT WORK FOR UNCOVER NOSE WELL BUT DECENT
    #GAB 1.3 19 Seems decent (PROB NOT RAB)
    
    # GAB
    # 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this
    masked = True

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        masked = False


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        masked = True
        cv2.putText(img,'Mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 0),4)#((x*2+w)/2,(y*2+h)/2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    # if haarcascade_frontalface_default.xml detects face but our didnt we put txt no mask
    if not masked:
        cv2.putText(img,'No mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),4)#((x*2+w)/2,(y*2+h)/2)
    iter_counter += 1 
    if masked:
        pos_counter += 1
    if iter_counter == 100:
        break
    print(str(iter_counter))
    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

time.sleep(10)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    """""
    Try diffrent Params in diff light / brightness / face size (if far or close)
    """
    masks = mask_cascade.detectMultiScale(gray,1.3,19) 
    #FOR RAB:
    #1.3, 15 WORK WELL
    
    #1.3 19 Seems best in tomer's conditions at night.
    #1.3,11 DOESNT WORK FOR UNCOVER NOSE WELL BUT DECENT
    #GAB 1.3 19 Seems decent (PROB NOT RAB)
    
    # GAB
    # 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this
    masked = True

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        masked = False


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        masked = True
        cv2.putText(img,'Mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 0),4)#((x*2+w)/2,(y*2+h)/2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    # if haarcascade_frontalface_default.xml detects face but our didnt we put txt no mask
    if not masked:
        cv2.putText(img,'No mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),4)#((x*2+w)/2,(y*2+h)/2)
    iter_counter += 1 
    if not masked:
        neg_counter += 1
    if iter_counter == 200:
        break
    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    print(str(iter_counter))

score = (neg_counter+pos_counter)/iter_counter
print(score)
cap.release()
cv2.destroyAllWindows()

Test ACC: 93

Fourth ATTEMT TO CREATE haar cascade classifier for faceMask detection
----------------------------------------------------------------------------------------------------------

    at this model we tried the following params
    files size increased to 224x224
    the training is on 24 width 24 height to match original VJ
    training methods changed:
    stages: 25
    mode: ALL
    neg img: 900
    pos img: 1800
    Boost type: RAB (Real Adaboost)

    TEST ACC (200 images): 0.97%

    clean the DS from curropted img
    if class_nam in ['face_with_mask',"gas_mask", "face_shield", "mask_surgical", "mask_colorful"]:
     changed to:
      if class_nam in ['face_with_mask', "mask_surgical", "mask_colorful"]:

    to pass bad images.

In [ ]:
images=[]
labels=[]

f=open('info.dat', 'w')
fb = open('bg.txt', 'w')
poscounter=1
negcounter=1
masked_img  = False
for filename in os.listdir(images_dir):
    num = filename.split('.')[ 0 ]
    print("loading image: {}".format(filename))
    if int(num) > 1800:
        class_name = None
        anno = filename + ".json"
        with open(os.path.join(anno_dir, anno)) as json_file:
            json_data = json.load(json_file)
            no_anno = json_data["NumOfAnno"]
            k = 0
            for i in range(0, no_anno):
                class_nam = json_data['Annotations'][i]['classname']
                if class_nam in ['face_with_mask', "mask_surgical", "mask_colorful"]:
                    class_name = 'face_with_mask'
                    k = i
                    masked_img = True
                    break
                elif class_nam in ['face_no_mask,"hijab_niqab', 'face_other_covering', "face_with_mask_incorrect", "scarf_bandana","balaclava_ski_mask", "other" ]:                   
                    class_name = 'face_no_mask'
                    k = i
                    masked_img = False
                    break
                else:
                    continue
                    
            box = json_data[ 'Annotations' ][k][ 'BoundingBox' ]
            (x1, x2, y1, y2) = box
            print('point:'+str(x1)+','+str(x2)+','+str(y1)+','+str(y2))
            # pathes on linux syntax since train is on linux.
            if masked_img:
                #f.write('pos/' + filename + ' 1 ' + str(x1) + ' ' + str(x2) + ' ' +  str(y1) + ' ' + str(y2) + ' \n')
                f.write('pos/' + str(poscounter) + ' 1 0 0 224 224 \n')
            else:
                fb.write('neg/' + str(negcounter) + ' \n')
        if class_name is not None: # we ignore some faces here
            image = cv2.imread(os.path.join(images_dir, filename),cv2.IMREAD_GRAYSCALE)
            img = image[x2:y2, x1:y1]
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(class_name)  
            if masked_img:
                if filename.split('.')[ 1 ] == 'jpeg':
                    print('pos\\'+str(poscounter)+"."+filename.split('.')[ 1 ])
                    cv2.imwrite('pos\\'+str(poscounter)+"."+'jpg',img)
                    poscounter += 1
                else:
                    print('pos\\'+str(poscounter)+"."+filename.split('.')[ 1 ])
                    cv2.imwrite('pos\\'+str(poscounter)+"."+filename.split('.')[ 1 ],img)
                    poscounter += 1
            else:
                if filename.split('.')[ 1 ] == 'jpeg':
                    print('neg\\'+str(negcounter)+"."+filename.split('.')[ 1 ])
                    cv2.imwrite('neg\\'+str(negcounter)+"."+'jpg',img)
                    negcounter += 1
                else:
                    print('neg\\'+str(negcounter)+"."+filename.split('.')[ 1 ])
                    cv2.imwrite('neg\\'+str(negcounter)+"."+filename.split('.')[ 1 ],img)
                    negcounter += 1

   
f.close()
fb.close()
images = np.array(images, dtype="float32")
labels = np.array(labels)
print(len(images))
print(len(labels))

    can pass this numerical info why i changed what i changed:

"""""
Conclusion:

cascade tarined as HAAR

-v-HAAR classifiers are very accurate but require a lot more time to train
-x-LBP classifiers on the other hand are less accurate but train much quicker and detect almost 3 times faster.

-mode <BASIC (default) | CORE | ALL> : Selects the type of Haar features set used in training. BASIC use only upright features, while ALL uses the full set of upright and 45 degree rotated feature set. See [148] for more details.
---> BASIC done, try ALL

Adaboost PARAMS
---> value + explain:

-acceptanceRatioBreakValue <break_value> : This argument is used to determine how precise your model should keep learning and when to stop. A good guideline is to train not further than 10e-5, to ensure the model does not overtrain on your training data. By default this value is set to -1 to disable this feature.
---> -1, we dont need this since we dont have unlimited time.

-Boost type: GAB (Gentle Adaboost)
for params explain: https://topic.alibabacloud.com/a/comparison-of-several-boost-algorithms-discrete-adaboost-real-adaboost-logitboost-gentle-adaboost-__-machine-learning_8_8_20288852.html
-weightTrimRate <weight_trim_rate> : Specifies whether trimming should be used and its weight. A decent choice is 0.95.
---> 0.95
-maxDepth <max_depth_of_weak_tree> : Maximal depth of a weak tree. A decent choice is 1, that is case of stumps.
---> 1
-maxFalseAlarmRate <max_false_alarm_rate> : Maximal desired false alarm rate for each stage of the classifier. Overall false alarm rate may be estimated as (max_false_alarm_rate ^ number_of_stages)
---> 0.5^15 / 0.5^25 (FA - 0.5, Stages - 15 / 25) small nums decent choise (may wanna try lower if i have more time)
-maxWeakCount <max_weak_tree_count> : Maximal count of weak trees for every cascade stage. The boosted classifier (stage) will have so many weak trees (<=maxWeakCount), as needed to achieve the given -maxFalseAlarmRate.
---> 100 by default choise (may wanna try more options if i have more time)

neg img: 1082
pos img: 69% * 2600

changed: 224x224 train pic the detection is 24x24

"""

Real time test with webcam:

In [ ]:

import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#this is the cascade we just made. Call what you want
mask_cascade = cv2.CascadeClassifier('mask_cascade_RAB_ALL.xml')

cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    masks = mask_cascade.detectMultiScale(gray,1.6,25)# 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this


    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)


    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

    solution felt good and accurate on real time therefore we evaluate our accuracy now :)

    we take 100 images with mask each image iter_counter+=1 if mask detect pos_counter += 1
    we take 100 images without mask each image iter_counter+=1 if mask detect neg_counter += 1
    then we give score by (pos_counter+neg_counter)/iter_counter ==> 0.97

In [ ]:
from typing import Counter
import numpy as np
import cv2
import time

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#this is the cascade we just made. Call what you want
mask_cascade = cv2.CascadeClassifier('mask_cascade_RAB_ALL.xml')

cap = cv2.VideoCapture(0)
pos_counter = 0
iter_counter = 0
neg_counter = 0

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    """""
    Try diffrent Params in diff light / brightness / face size (if far or close)
    """
    masks = mask_cascade.detectMultiScale(gray,1.3,8) 
    #FOR RAB:
    #1.3, 15 WORK WELL
    
    #1.3 19 Seems best in tomer's conditions at night.
    #1.3,11 DOESNT WORK FOR UNCOVER NOSE WELL BUT DECENT
    #GAB 1.3 19 Seems decent (PROB NOT RAB)
    
    # GAB
    # 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this
    masked = True

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        masked = False


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        masked = True
        cv2.putText(img,'Mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 0),4)#((x*2+w)/2,(y*2+h)/2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    # if haarcascade_frontalface_default.xml detects face but our didnt we put txt no mask
    if not masked:
        cv2.putText(img,'No mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),4)#((x*2+w)/2,(y*2+h)/2)
    iter_counter += 1 
    if masked:
        pos_counter += 1
    if iter_counter == 100:
        break
    print(str(iter_counter))
    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

time.sleep(10)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    """""
    Try diffrent Params in diff light / brightness / face size (if far or close)
    """
    masks = mask_cascade.detectMultiScale(gray,1.3,19) 
    #FOR RAB:
    #1.3, 15 WORK WELL
    
    #1.3 19 Seems best in tomer's conditions at night.
    #1.3,11 DOESNT WORK FOR UNCOVER NOSE WELL BUT DECENT
    #GAB 1.3 19 Seems decent (PROB NOT RAB)
    
    # GAB
    # 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this
    masked = True

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        masked = False


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        masked = True
        cv2.putText(img,'Mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 0),4)#((x*2+w)/2,(y*2+h)/2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    # if haarcascade_frontalface_default.xml detects face but our didnt we put txt no mask
    if not masked:
        cv2.putText(img,'No mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),4)#((x*2+w)/2,(y*2+h)/2)
    iter_counter += 1 
    if not masked:
        neg_counter += 1
    if iter_counter == 200:
        break
    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    print(str(iter_counter))

score = (neg_counter+pos_counter)/iter_counter
print(score)
cap.release()
cv2.destroyAllWindows()

TEST ACC: 0.97

    WAYS TO IMPROVE:
    train on more resized images (same images diff sizes to fit any camera at any distance) and with more angles to increase dataset 
    and add more random neg images, 
    try another boots method ( LB, etc),
    try detecting edges with cv erode and canny and see if it increase our accuarcy,
    try basic Haar train (the xml shows it as basic and also haar_cascade it so its not reliable to think its basic all applies 
    45 angles and more features so i might wanna use it on my dataset)
    adjust the brightness of the picture according to day/night so the params of the cascade will work better on more situations

Fifth ATTEMT TO CREATE haar cascade classifier for faceMask detection
----------------------------------------------------------------------------------------------------------

model name: RAB211Tree

    CHANGES:
    files size increased to 224x224
    the training is on 24 width 24 height to match original VJ
    training methods changed:
    mode: ALL
    neg img: 1000
    pos img: 2000
    Boost type: RAB (Real Adaboost)
    Max tree depth: 211 (like VJ)

In [ ]:

import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#this is the cascade we just made. Call what you want
mask_cascade = cv2.CascadeClassifier('haarcascade_facemask_RAB211Tree.xml')

cap = cv2.VideoCapture(0)
isPrevMasked = True
masked = True
while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    
    # add this
    # image, reject levels level weights.
    """""
    Try diffrent Params in diff light / brightness / face size (if far or close)
    """
    masks = mask_cascade.detectMultiScale(gray,1.34,18) #1.34,18/1.33,16 seems good at 20:00 yellow light
    # 1.34,18
    #FOR RAB:
    #1.3, 15 WORK WELL
    
    #1.3 19 Seems best in tomer's conditions at night.
    #1.3,11 DOESNT WORK FOR UNCOVER NOSE WELL BUT DECENT
    #GAB 1.3 19 Seems decent (PROB NOT RAB)
    
    # GAB
    # 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this
    #masked = True
    masked = True
    for (x,y,w,h) in faces:
        if not isPrevMasked:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        masked = False



        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        masked = True
        cv2.putText(img,'Mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 0),4)#((x*2+w)/2,(y*2+h)/2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    # if haarcascade_frontalface_default.xml detects face but our didnt we put txt no mask
    if (not isPrevMasked) and (not masked):
        cv2.putText(img,'No mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),4)#((x*2+w)/2,(y*2+h)/2)


    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    if masked:
        isPrevMasked = True
    else:
        isPrevMasked = False

cap.release()
cv2.destroyAllWindows()

    evaluate the model:

In [ ]:
from typing import Counter
import numpy as np
import cv2
import time

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#this is the cascade we just made. Call what you want
mask_cascade = cv2.CascadeClassifier('haarcascade_facemask_RAB211Tree.xml')

cap = cv2.VideoCapture(0)
pos_counter = 0
iter_counter = 0
neg_counter = 0

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    """""
    Try diffrent Params in diff light / brightness / face size (if far or close)
    """
    masks = mask_cascade.detectMultiScale(gray,1.34,18) 
    #FOR RAB:
    #1.3, 15 WORK WELL
    
    #1.3 19 Seems best in tomer's conditions at night.
    #1.3,11 DOESNT WORK FOR UNCOVER NOSE WELL BUT DECENT
    #GAB 1.3 19 Seems decent (PROB NOT RAB)
    
    # GAB
    # 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this
    masked = True

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        masked = False


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        masked = True
        cv2.putText(img,'Mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 0),4)#((x*2+w)/2,(y*2+h)/2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    # if haarcascade_frontalface_default.xml detects face but our didnt we put txt no mask
    if not masked:
        cv2.putText(img,'No mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),4)#((x*2+w)/2,(y*2+h)/2)
    iter_counter += 1 
    if masked:
        pos_counter += 1
    if iter_counter == 100:
        break
    print(str(iter_counter))
    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

time.sleep(10)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    """""
    Try diffrent Params in diff light / brightness / face size (if far or close)
    """
    masks = mask_cascade.detectMultiScale(gray,1.3,19) 
    #FOR RAB:
    #1.3, 15 WORK WELL
    
    #1.3 19 Seems best in tomer's conditions at night.
    #1.3,11 DOESNT WORK FOR UNCOVER NOSE WELL BUT DECENT
    #GAB 1.3 19 Seems decent (PROB NOT RAB)
    
    # GAB
    # 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this
    masked = True

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        masked = False


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        masked = True
        cv2.putText(img,'Mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 0),4)#((x*2+w)/2,(y*2+h)/2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    # if haarcascade_frontalface_default.xml detects face but our didnt we put txt no mask
    if not masked:
        cv2.putText(img,'No mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),4)#((x*2+w)/2,(y*2+h)/2)
    iter_counter += 1 
    if not masked:
        neg_counter += 1
    if iter_counter == 200:
        break
    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    print(str(iter_counter))

score = (neg_counter+pos_counter)/iter_counter
print(score)
cap.release()
cv2.destroyAllWindows()

TEST ACC: 0.99

    HOW TO IMPROVE:
    train on more resized images (same images diff sizes to fit any camera at any distance) and with more angles to increase dataset 
    and add more random neg images, 
    try another boots method ( LB, etc),
    try detecting edges with cv erode and canny and see if it increase our accuarcy,
    try basic Haar train (the xml shows it as basic and also haar_cascade it so its not reliable to think its basic all applies 
    45 angles and more features so i might wanna use it on my dataset)
    adjust the brightness of the picture according to day/night so the params of the cascade will work better on more situations

    i felt like trying other adaboost type and finish after this (12 days of work)

Sixth ATTEMT TO CREATE haar cascade classifier for faceMask detection
----------------------------------------------------------------------------------------------------------

model name: GAB211Tree

    CHANGES:
    V1
    Max depth tree count changed to 211 to fit VJ
    training methods changed:
    mode: Basic (fit VJ according to haar_classifiers*.xml)
    neg img: 1000
    pos img: 2000
    Boost type: GAB (Gentle Adaboost)
    depth tree 211

In [ ]:
import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#this is the cascade we just made. Call what you want
mask_cascade = cv2.CascadeClassifier('haarcascade_face_mask.xml')

cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    """""
    Try diffrent Params in diff light / brightness / face size (if far or close)
    """
    masks = mask_cascade.detectMultiScale(gray,1.34,18) 
    #FOR RAB:
    #1.3, 15 WORK WELL
    
    #1.3 19 Seems best in tomer's conditions at night.
    #1.3,11 DOESNT WORK FOR UNCOVER NOSE WELL BUT DECENT
    #GAB 1.3 19 Seems decent (PROB NOT RAB)
    
    # GAB
    # 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this
    masked = True

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        masked = False


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        masked = True
        cv2.putText(img,'Mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 0),4)#((x*2+w)/2,(y*2+h)/2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    # if haarcascade_frontalface_default.xml detects face but our didnt we put txt no mask
    if not masked:
        cv2.putText(img,'No mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),4)#((x*2+w)/2,(y*2+h)/2)


    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

Evaluate the model

In [ ]:
from typing import Counter
import numpy as np
import cv2
import time

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#this is the cascade we just made. Call what you want
mask_cascade = cv2.CascadeClassifier('haarcascade_face_mask.xml')

cap = cv2.VideoCapture(0)
pos_counter = 0
iter_counter = 0
neg_counter = 0

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    """""
    Try diffrent Params in diff light / brightness / face size (if far or close)
    """
    masks = mask_cascade.detectMultiScale(gray,1.34,18) 
    #FOR RAB:
    #1.3, 15 WORK WELL
    
    #1.3 19 Seems best in tomer's conditions at night.
    #1.3,11 DOESNT WORK FOR UNCOVER NOSE WELL BUT DECENT
    #GAB 1.3 19 Seems decent (PROB NOT RAB)
    
    # GAB
    # 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this
    masked = True

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        masked = False


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        masked = True
        cv2.putText(img,'Mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 0),4)#((x*2+w)/2,(y*2+h)/2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    # if haarcascade_frontalface_default.xml detects face but our didnt we put txt no mask
    if not masked:
        cv2.putText(img,'No mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),4)#((x*2+w)/2,(y*2+h)/2)
    iter_counter += 1 
    if masked:
        pos_counter += 1
    if iter_counter == 100:
        break
    print(str(iter_counter))
    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

time.sleep(10)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # add this
    # image, reject levels level weights.
    """""
    Try diffrent Params in diff light / brightness / face size (if far or close)
    """
    masks = mask_cascade.detectMultiScale(gray,1.3,19) 
    #FOR RAB:
    #1.3, 15 WORK WELL
    
    #1.3 19 Seems best in tomer's conditions at night.
    #1.3,11 DOESNT WORK FOR UNCOVER NOSE WELL BUT DECENT
    #GAB 1.3 19 Seems decent (PROB NOT RAB)
    
    # GAB
    # 1.6,30 224x224 (24x24)   # 1.3,20
    
    # add this
    masked = True

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        masked = False


        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]    
    for (x,y,w,h) in masks:
        masked = True
        cv2.putText(img,'Mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 0),4)#((x*2+w)/2,(y*2+h)/2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    # if haarcascade_frontalface_default.xml detects face but our didnt we put txt no mask
    if not masked:
        cv2.putText(img,'No mask',(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255),4)#((x*2+w)/2,(y*2+h)/2)
    iter_counter += 1 
    if not masked:
        neg_counter += 1
    if iter_counter == 200:
        break
    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    print(str(iter_counter))

score = (neg_counter+pos_counter)/iter_counter
print(score)
cap.release()
cv2.destroyAllWindows()

TEST ACC: 0.965

TEST ACC: 1.0

Avg ACC: 0.9825

we tested first got 0.965 then again on another 200 photos and got 1.0

    HOW TO IMPROVE:
    train on more resized images (same images diff sizes to fit any camera at any distance) and with more angles to increase dataset and add more random neg images, 
    try another boots method (LB, etc),
    try detecting edges with cv erode and canny and see if it increase our accuarcy,
    train on bigger samples - Req alot of RAM
